In [46]:
import pandas as pd
import numpy as np
import utilities.misc as um
import utilities.constants as uc
import pdb
from blp.bdx import bdh,bdp,bds
from fql.fql import Factset_Query
from fql.util import bbg_to_fs, fs_to_bbg,fql_date
from blp.util import get_bbg_usual_col, group_marcap,get_ashare_exchange,load_compo
import feather
import os

#plotly related
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import utilities.display as ud

from datetime import datetime

import utilities.mathematics as umath

from joblib import load, dump

import matplotlib.pyplot as plt

import datatable as dt
from datatable import f,by,sort,join,shift,ifelse
dt.init_styles()

In [2]:
path=uc.root_path_data+'\\connect\ccass_all\\'
universe=pd.read_excel(path+'full_eqs_hk_static.xlsx')

cols_to_numeric=['marcap_busd','adv_3m_musd','float']
for col in cols_to_numeric:
    universe[col]=pd.to_numeric(universe[col])
    
cols_to_na=list(set(universe.columns.tolist())-set(cols_to_numeric))
for col in cols_to_na:
    universe[col]=universe[col].fillna('na')
    

In [3]:
# # get some bbg data
# px_data=bdh(universe['ticker'].tolist(),['PX_LAST'],datetime(2022,1,1),um.today_date(),adjustmentFollowDPDF=False)
# feather.write_dataframe(px_data.reset_index(),path+'px_data.feather')

In [4]:
# # collect and dump
# files_all=um.iterate_csv(path+'data_raw\\',iterate_others=[True,'.feather'])
# collector=[]
# for file_i in files_all:
#     collector.append(feather.read_dataframe(path+'data_raw//%s.feather' % (file_i)))
# data_raw=pd.concat(collector)
# feather.write_dataframe(data_raw,path+'data_raw.feather')

In [5]:
# some datatable operation
data_raw_dt=dt.Frame(feather.read_dataframe(path+'data_raw.feather'))
universe_dt=dt.Frame(universe)
px_data_dt=dt.Frame(feather.read_dataframe(path+'px_data.feather'))

In [6]:
# calculate some stats

universe_dt.key=['ticker']
data=data_raw_dt[:,:,join(universe_dt)]

px_data_dt.key=['date','ticker']
data=data[:,:,join(px_data_dt)]

data['hld_value_musd']=data[:,f['PX_LAST']*f['shares']/7.85/1000000]
data=data[:,:,sort(['ticker','ccass_id','ccass_name','date'])]
data=data[:,f[:].extend({'hld_value_musd_chg':(f['shares']-dt.shift(f['shares']))*f['PX_LAST']/7.85/1000000 }), by(['ticker','ccass_id','ccass_name']) ]

size_quintile=data[:,dt.last(f['marcap_busd']),by('ticker')][:,f[:].extend({'size_q':dt.qcut(f['marcap_busd'],nquantiles=10)})]
del size_quintile[:,'marcap_busd']
size_quintile.key=['ticker']
data=data[:,:,join(size_quintile)]
data['size_q']=data[:,f['size_q']+1]

data['hld_value_musd_chg']=ifelse(f['hld_value_musd_chg']==None,0,f['hld_value_musd_chg']) 
# this can also be used through by, e.g. data[:,ifelse(f['hld_value_musd_chg']==None,0,f['hld_value_musd_chg']),by(['ccass_id','date'])]
data['is_hld_chg']=ifelse(f['hld_value_musd_chg']==0,0,1)




# Some other quite interesting func
- datatable.str.split_into_nhot
- the g func https://datatable.readthedocs.io/en/latest/api/dt/join.html
- print(data.types)
- convert type https://datatable.readthedocs.io/en/latest/api/dt/as_type.html

In [7]:
ccass_freq=data[:,{'freq':dt.sum(f['is_hld_chg'])/dt.count(f['is_hld_chg']),'ccass_id':dt.last(f['ccass_id'])},by(['ccass_name','ticker'])].to_pandas()

In [30]:
data.stypes

(stype.str32,
 stype.str32,
 stype.str32,
 stype.float64,
 stype.float64,
 stype.time64,
 stype.str32,
 stype.float64,
 stype.float64,
 stype.float64,
 stype.str32,
 stype.str32,
 stype.str32,
 stype.float64,
 stype.float64,
 stype.float64,
 stype.int32,
 stype.int32)

In [45]:
print(data.types)

[Type.str32, Type.str32, Type.str32, Type.float64, Type.float64, Type.time64, Type.str32, Type.float64, Type.float64, Type.float64, Type.str32, Type.str32, Type.str32, Type.float64, Type.float64, Type.float64, Type.int32, Type.int32]


In [47]:
data

,ticker,ccass_id,ccass_name,shares,stake,date,name,marcap_busd,adv_3m_musd,float,…,PX_LAST,hld_value_musd,hld_value_musd_chg,size_q,is_hld_chg
,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪,▪▪▪▪
0,1 HK Equity,,CHAN YIM TAO MARGARET *,10000,0,2022-11-25T00:00:00,CK HUTCHISON HOLDINGS LTD,19.9282,21.519,69.3397,…,43.55,0.0554777,0,10,0
1,1 HK Equity,,CHAN YIM TAO MARGARET *,10000,0,2022-12-02T00:00:00,CK HUTCHISON HOLDINGS LTD,19.9282,21.519,69.3397,…,44,0.056051,0,10,0
2,1 HK Equity,,CHAN YIM TAO MARGARET *,10000,0,2022-12-09T00:00:00,CK HUTCHISON HOLDINGS LTD,19.9282,21.519,69.3397,…,45.45,0.0578981,0,10,0
3,1 HK Equity,,CHAN YIM TAO MARGARET *,10000,0,2022-12-16T00:00:00,CK HUTCHISON HOLDINGS LTD,19.9282,21.519,69.3397,…,45.45,0.0578981,0,10,0
4,1 HK Equity,,CHAN YIM TAO MARGARET *,10000,0,2022-12-23T00:00:00,CK HUTCHISON HOLDINGS LTD,19.9282,21.519,69.3397,…,46.2,0.0588535,0,10,0
5,1 HK Equity,,CHAN YIM TAO MARGARET *,10000,0,2022-12-30T00:00:00,CK HUTCHISON HOLDINGS LTD,19.9282,21.519,69.3397,…,46.85,0.0596815,0,10,0
6,1 HK Equity,,CHAN YIM TAO MARGARET *,10000,0,2023-01-06T00:00:00,CK HUTCHISON HOLDINGS LTD,19.9282,21.519,69.3397,…,48.55,0.0618471,0,10,0
7,1 HK Equity,,CHAN YIM TAO MARGARET *,10000,0,2023-01-13T00:00:00,CK HUTCHISON HOLDINGS LTD,19.9282,21.519,69.3397,…,48.85,0.0622293,0,10,0
8,1 HK Equity,,CHAN YIM TAO MARGARET *,10000,0,2023-01-20T00:00:00,CK HUTCHISON HOLDINGS LTD,19.9282,21.519,69.3397,…,49.7,0.0633121,0,10,0
